## Analysis pilot data Sysmex (BloodCounts Project)

**Background**: Pilot data is provided from a cohort of COVID patients at the VUMC location, some with pulmonary embolisms. Data is obtained from blood sampling and analysed using Sysmex software. In addition to the regular full blood count, additional information is provided after decryption by Sysmex including flow cytometry data. Respective data is of the cohort is retrieved after permission of lead investigators and extracted based on SampleIds via the Research Data Platform overseen by the Business Intellegence Department (jira ticket:VUBI-8429). Sample data was subsequently extracted by the TraiL in .116 files before decryption by Sysmex.

**Analysis**: Perform exploratory data analysis of the blood counts data, including FCS files, and investigate predictive value for lung embolism.

**Date**:        23-01-2024

**Author**:      Stephan van der Zwaard

In [2]:
# Load required libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import flowkit as fk

# Set options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
# Check if CSV can be found:
os.path.exists(os.getcwd()+'/testfiles_AMS/XN_SAMPLE.csv')

True

In [5]:
# Read CSV data
data = pd.read_csv(os.getcwd()+'/testfiles_AMS/XN_SAMPLE.csv')

In [6]:
# Inspect dataframe (221 records by 492 cols)
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Columns: 492 entries, Nickname to Unnamed: 491
dtypes: float64(279), int64(39), object(174)
memory usage: 849.6+ KB


,Nickname,Analyzer ID,Date,Time,Rack,Position,Sample No.,Sample Inf.,Order Type,Reception Date,Measurement Mode,Discrete,Patient ID,Analysis Info.,Judgment,Positive(Diff.),Positive(Morph.),Positive(Count),Error(Func.),Error(Result),Order Info.,WBC Abnormal,WBC Suspect,RBC Abnormal,RBC Suspect,PLT Abnormal,PLT Suspect,WBC Info.,PLT Info.,Rule Result,Validate,Validator,Action Message (Check),Action Message (Review),Action Message (Retest),Sample Comment,Patient Name,Birth,Sex,Patient Comment,Ward Name,Doctor Name,Output,Sequence No.,IP ABN(WBC)WBC Abn Scattergram,IP ABN(WBC)Neutropenia,IP ABN(WBC)Neutrophilia,IP ABN(WBC)Lymphopenia,IP ABN(WBC)Lymphocytosis,IP ABN(WBC)Monocytosis,IP ABN(WBC)Eosinophilia,IP ABN(WBC)Basophilia,IP ABN(WBC)Leukocytopenia,IP ABN(WBC)Leukocytosis,IP ABN(WBC)NRBC Present,IP ABN(WBC)IG Present,IP ABN(RBC)RBC Abn Distribution,IP ABN(RBC)Dimorphic Population,IP ABN(RBC)Anisocytosis,IP ABN(RBC)Microcytosis,IP ABN(RBC)Macrocytosis,IP ABN(RBC)Hypochromia,IP ABN(RBC)Anemia,IP ABN(RBC)Erythrocytosis,IP ABN(RBC)RET Abn Scattergram,IP ABN(RBC)Reticulocytosis,IP ABN(PLT)PLT Abn Distribution,IP ABN(PLT)Thrombocytopenia,IP ABN(PLT)Thrombocytosis,IP ABN(PLT)PLT Abn Scattergram,IP SUS(WBC)Blasts/Abn Lympho?,IP SUS(WBC)Blasts?,IP SUS(WBC)Abn Lympho?,IP SUS(WBC)Left Shift?,IP SUS(WBC)Atypical Lympho?,IP SUS(RBC)RBC Agglutination?,IP SUS(RBC)Turbidity/HGB Interf?,IP SUS(RBC)Iron Deficiency?,IP SUS(RBC)HGB Defect?,IP SUS(RBC)Fragments?,IP SUS(PLT)PLT Clumps?,Q-Flag(Blasts/Abn Lympho?),Q-Flag(Blasts?),Q-Flag(Abn Lympho?),Q-Flag(Left Shift?),Q-Flag(Atypical Lympho?),Q-Flag(RBC Agglutination?),Q-Flag(Turbidity/HGB Interf?),Q-Flag(Iron Deficiency?),Q-Flag(HGB Defect?),Q-Flag(Fragments?),Q-Flag(PLT Clumps?),WBC(10^3/uL),WBC/M,RBC(10^6/uL),RBC/M,HGB(g/dL),HGB/M,HCT(%),HCT/M,MCV(fL),MCV/M,MCH(pg),MCH/M,MCHC(g/dL),MCHC/M,PLT(10^3/uL),PLT/M,RDW-SD(fL),RDW-SD/M,RDW-CV(%),RDW-CV/M,PDW(fL),PDW/M,MPV(fL),MPV/M,P-LCR(%),P-LCR/M,PCT(%),PCT/M,NRBC#(10^3/uL),NRBC#/M,NRBC%(%),NRBC%/M,NEUT#(10^3/uL),NEUT#/M,LYMPH#(10^3/uL),LYMPH#/M,MONO#(10^3/uL),MONO#/M,EO#(10^3/uL),EO#/M,BASO#(10^3/uL),BASO#/M,NEUT%(%),NEUT%/M,LYMPH%(%),LYMPH%/M,MONO%(%),MONO%/M,EO%(%),EO%/M,BASO%(%),BASO%/M,IG#(10^3/uL),IG#/M,IG%(%),IG%/M,RET%(%),RET%/M,RET#(10^6/uL),RET#/M,IRF(%),IRF/M,LFR(%),LFR/M,MFR(%),MFR/M,HFR(%),HFR/M,RET-He(pg),RET-He/M,IPF(%),IPF/M,[PLT-I(10^3/uL)],[PLT-I/M],MicroR(%),MicroR/M,MacroR(%),MacroR/M,[TNC(10^3/uL)],[TNC/M],[WBC-N(10^3/uL)],[WBC-N/M],[TNC-N(10^3/uL)],[TNC-N/M],[BA-N#(10^3/uL)],[BA-N#/M],[BA-N%(%)],[BA-N%/M],[WBC-D(10^3/uL)],[WBC-D/M],[TNC-D(10^3/uL)],[TNC-D/M],[NEUT#&(10^3/uL)],[NEUT#&/M],[NEUT%&(%)],[NEUT%&/M],[LYMP#&(10^3/uL)],[LYMP#&/M],[LYMP%&(%)],[LYMP%&/M],[HFLC#(10^3/uL)],[HFLC#/M],[HFLC%(%)],[HFLC%/M],[BA-D#(10^3/uL)],[BA-D#/M],[BA-D%(%)],[BA-D%/M],[NE-SSC(ch)],[NE-SSC/M],[NE-SFL(ch)],[NE-SFL/M],[NE-FSC(ch)],[NE-FSC/M],[LY-X(ch)],[LY-X/M],[LY-Y(ch)],[LY-Y/M],[LY-Z(ch)],[LY-Z/M],[MO-X(ch)],[MO-X/M],[MO-Y(ch)],[MO-Y/M],[MO-Z(ch)],[MO-Z/M],[NE-WX],[NE-WX/M],[NE-WY],[NE-WY/M],[NE-WZ],[NE-WZ/M],[LY-WX],[LY-WX/M],[LY-WY],[LY-WY/M],[LY-WZ],[LY-WZ/M],[MO-WX],[MO-WX/M],[MO-WY],[MO-WY/M],[MO-WZ],[MO-WZ/M],[WBC-P(10^3/uL)],[WBC-P/M],[TNC-P(10^3/uL)],[TNC-P/M],[RBC-O(10^6/uL)],[RBC-O/M],[PLT-O(10^3/uL)],[PLT-O/M],RBC-He(pg),RBC-He/M,Delta-He(pg),Delta-He/M,[RET-Y(ch)],[RET-Y/M],[RET-RBC-Y(ch)],[RET-RBC-Y/M],[IRF-Y(ch)],[IRF-Y/M],[FRC#(10^6/uL)],[FRC#/M],[FRC%(%)],[FRC%/M],HYPO-He(%),HYPO-He/M,HYPER-He(%),HYPER-He/M,[RPI],[RPI/M],[RET-UPP],[RET-UPP/M],[RET-TNC],[RET-TNC/M],[PLT-F(10^3/uL)],[PLT-F/M],[H-IPF(%)],[H-IPF/M],IPF#(10^3/uL),IPF#/M,WBC-BF(10^3/uL),WBC-BF/M,RBC-BF(10^6/uL),RBC-BF/M,MN#(10^3/uL),MN#/M,PMN#(10^3/uL),PMN#/M,MN%(%),MN%/M,PMN%(%),PMN%/M,TC-BF#(10^3/uL),TC-BF#/M,[HF-BF#(10^3/uL)],[HF-BF#/M],[HF-BF%(/100WBC)],[HF-BF%/M],[NE-BF#(10^3/uL)],[NE-BF#/M],[NE-BF%(%)],[NE-BF%/M],[LY-BF#(10^3/uL)],[LY-BF#/M],[LY-BF%(%)],[LY-BF%/M],[MO-BF#(10^3/uL)],[MO-BF#/M],[MO-BF%(%)],[MO-BF%/M],[EO-BF#(10^3/uL)],[EO-BF#/M],[EO-BF%(%)],[EO-BF%/

In [7]:
# Inspect dataframe (318 vars)
print(data.columns.values)

['Nickname' 'Analyzer ID' 'Date' 'Time' 'Rack' 'Position' 'Sample No.'
 'Sample Inf.' 'Order Type' 'Reception Date' 'Measurement Mode' 'Discrete'
 'Patient ID' 'Analysis Info.' 'Judgment' 'Positive(Diff.)'
 'Positive(Morph.)' 'Positive(Count)' 'Error(Func.)' 'Error(Result)'
 'Order Info.' 'WBC Abnormal' 'WBC Suspect' 'RBC Abnormal' 'RBC Suspect'
 'PLT Abnormal' 'PLT Suspect' 'WBC Info.' 'PLT Info.' 'Rule Result'
 'Validate' 'Validator' 'Action Message (Check)' 'Action Message (Review)'
 'Action Message (Retest)' 'Sample Comment' 'Patient Name' 'Birth' 'Sex'
 'Patient Comment' 'Ward Name' 'Doctor Name' 'Output' 'Sequence No.'
 'IP ABN(WBC)WBC Abn Scattergram' 'IP ABN(WBC)Neutropenia'
 'IP ABN(WBC)Neutrophilia' 'IP ABN(WBC)Lymphopenia'
 'IP ABN(WBC)Lymphocytosis' 'IP ABN(WBC)Monocytosis'
 'IP ABN(WBC)Eosinophilia' 'IP ABN(WBC)Basophilia'
 'IP ABN(WBC)Leukocytopenia' 'IP ABN(WBC)Leukocytosis'
 'IP ABN(WBC)NRBC Present' 'IP ABN(WBC)IG Present'
 'IP ABN(RBC)RBC Abn Distribution' 'IP ABN(RBC

In [9]:
# Check different measurement methods and corresponding counts
data['Measurement Mode'].unique()
data.groupby(by = ['Measurement Mode'])['Measurement Mode'].count()

Measurement Mode
LW      1
WB    220
Name: Measurement Mode, dtype: int64

In [10]:
# Check different measurement methods and corresponding counts
data['Discrete'].unique()
data.groupby(by = ['Discrete'])['Discrete'].count()

Discrete
CBC                21
CBC+DIFF          169
CBC+DIFF+PLT-F      2
CBC+DIFF+WPC       24
FREE SELECT         5
Name: Discrete, dtype: int64

In [11]:
plt.figure()


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [12]:
data.isnull().sum().sort_values(ascending=False) #/len(data) >0.9

Nickname                            221
HYPER-He/M                          221
[WBC-P/M]                           221
[TNC-P/M]                           221
[RBC-O/M]                           221
[PLT-O/M]                           221
RBC-He/M                            221
Delta-He/M                          221
[RET-Y/M]                           221
[RET-RBC-Y/M]                       221
[IRF-Y/M]                           221
[FRC#/M]                            221
[FRC%/M]                            221
HYPO-He/M                           221
[RPI/M]                             221
NRBC#/M                             221
[RET-UPP/M]                         221
[RET-TNC/M]                         221
[PLT-F/M]                           221
WBC-BF/M                            221
RBC-BF/M                            221
MN#/M                               221
PMN#/M                              221
MN%/M                               221
PMN%/M                              221


In [13]:
#Check missing data
missing    = data.isnull().sum().sort_values(ascending=False)
df_missing = pd.DataFrame({'missing': missing})
df_missing['perc']  = df_missing['missing']/len(data)
df_missing['rowid'] = df_missing.reset_index().index
df_missing

,missing,perc,rowid
Nickname,221,1.000000,0
HYPER-He/M,221,1.000000,1
[WBC-P/M],221,1.000000,2
[TNC-P/M],221,1.000000,3
[RBC-O/M],221,1.000000,4
[PLT-O/M],221,1.000000,5
RBC-He/M,221,1.000000,6
Delta-He/M,221,1.000000,7
[RET-Y/M],221,1.000000,8
[RET-RBC-Y/M],221,1.000000,9
